In [1]:
from google.colab import drive
drive.mount('/root/gdrive')


Mounted at /root/gdrive


In [2]:
!pip install transformers
!pip install tqdm
!pip install nltk
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=4fc82e723937e4dbefdc1fc0aa1656e7e447174acb72d0a4b595af557fb8e267
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score

import numpy as np

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import torch



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
pip install openprompt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99


In [6]:
cd /root/gdrive/MyDrive/OpenPrompt

/root/gdrive/MyDrive/OpenPrompt


In [7]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 12.8 MB/s eta 0:00:00


In [ ]:
from openprompt.data_utils.text_classification_dataset import AgnewsProcessor  # load specific news processor
from openprompt.data_utils.data_sampler import FewShotSampler # few shot
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate
from openprompt.prompts import ManualVerbalizer
from openprompt import PromptDataLoader
from openprompt.prompts import SoftVerbalizer
from openprompt import PromptForClassification
import torch
from transformers import  AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from openprompt.data_utils.text_classification_dataset import ToxicityProcessor


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [ ]:
dataset = {}
# trainvalid_dataset = processor.get_train_examples(dataset_path)
dataset['train'] = ToxicityProcessor().get_train_examples("/root/gdrive/MyDrive/toxicity_data")
#sampler  = FewShotSampler(num_examples_per_label=3, num_examples_per_label_dev=3, also_sample_dev=True)
#dataset['train'], dataset['validation'] = sampler(dataset['train'])
dataset['test'] = ToxicityProcessor().get_test_examples("/root/gdrive/MyDrive/toxicity_data")
print(len(dataset['train']))
print(len(dataset['test']))
print(dataset['train'][0])


51182
12796
{
  "guid": "83aa72a87f749fdc",
  "label": 1,
  "meta": {},
  "text_a": "lennie injured remainder season gettin prono three fucked brown crooked cant get game alnwick never get game spur fucked lawrence club going ahhahahahah fuck porno three hate scum hope mother saw knocking bint bunch filthy pikey bastard",
  "text_b": "",
  "tgt_text": null
}



In [ ]:
classes = ['0','1']


In [ ]:
#plm, tokenizer, model_config, WrapperClass=load_plm("bert", "bert-base-cased") # load pretrained model
#plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")
plm, tokenizer, model_config, WrapperClass=load_plm("opt", "facebook/opt-1.3b")


In [ ]:
# define the template
#template = ManualTemplate(tokenizer=tokenizer, text= '{"placeholder":"text_a"} It was {"mask"}',)
#template = ManualTemplate(tokenizer=tokenizer, text= 'What is the toxicity label of {"placeholder":"text_a"}? It was {"mask"}',)
#template = ManualTemplate(tokenizer=tokenizer, text= 'Analyzing the comment {"placeholder":"text_a"}, it can be determined that the toxicity label is {"mask"}',)
verbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        #"0": ['non-toxic'],
        "0": ['non-toxic'],
        "1": ['toxic'],
    },
    tokenizer = tokenizer,
)

#verbalizer = SoftVerbalizer(tokenizer, plm, num_classes=2)

#wrapped_example = template.wrap_one_example(dataset['train'][0])
#print('wrapped_example:',wrapped_example)


In [ ]:
use_cuda = True
# build the prompt model
prompt_model = PromptForClassification(plm=plm,template=None, verbalizer=verbalizer, freeze_plm=False)
if use_cuda:
    prompt_model = prompt_model.cuda()


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get sentence embeddings
train_sentences = [ex.text_a for ex in dataset['train']]
train_embeddings = model.encode(train_sentences)

test_sentences = [ex.text_a for ex in dataset['test']]
test_embeddings = model.encode(test_sentences)


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:


# Load the CSV file
df = pd.read_csv('/root/gdrive/MyDrive/new_toxicity_train/new_train.csv')

# Initialize SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get sentence embeddings
new_train_sentences = df['comment_text'].apply(str).tolist() # Ensuring all are strings
new_train_embeddings = model.encode(new_train_sentences)

# Get the K values
new_train_K_values = df['K'].tolist()

# Initialize the classifier
#classifier = RandomForestClassifier(n_estimators=100, random_state=42)
#classifier = LogisticRegression(random_state=42)
classifier = SVC(kernel='linear', random_state=42)

# Fit the classifier
classifier.fit(new_train_embeddings, new_train_K_values)


SVC(kernel='linear', random_state=42)

In [ ]:
pip install joblib


In [ ]:
from joblib import dump

# Save the trained classifier
dump(classifier, '/root/gdrive/MyDrive/apl_prompt_code/trained_model.joblib')


['/root/gdrive/MyDrive/apl_prompt_code/trained_model.joblib']

In [ ]:
from joblib import load
classifier = load('/root/gdrive/MyDrive/apl_prompt_code/trained_model.joblib')

In [ ]:
# Predict K values for test set
predicted_K = classifier.predict(test_embeddings)


In [ ]:
from scipy.spatial import KDTree

tree = KDTree(train_embeddings)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
allpreds = []
alllabels = []

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

# build the prompt model
prompt_model = PromptForClassification(plm=plm, template=None, verbalizer=verbalizer, freeze_plm=False)
if use_cuda:
    prompt_model = prompt_model.cuda()

max_seq_length = 2048

for idx, test_example in enumerate(tqdm(dataset['test'], desc="Predicting", unit="batch")):
    test_sentence = test_example.text_a
    test_embedding = model.encode([test_sentence])[0]  # Get the embedding of the test_sentence

    # predict the best K for this sentence
    best_k = classifier.predict(test_embedding.reshape(1, -1))[0]

    # find the top K most similar train sentences
    #tree = KDTree(train_embeddings)
    _, top_k_indices = tree.query(test_embedding.reshape(1, -1), k=best_k)

    # get the top K most similar examples
    top_k_train_examples = [dataset['train'][i] for i in top_k_indices[0]]

    # truncate top_k_train_examples and create template, to ensure not exceeed the length
    few_shot_text = ""
    for ex in top_k_train_examples:
        temp_text = f"{ex.text_a} It was {verbalizer.label_words[ex.label][0]}."
        temp_text = temp_text if len(temp_text) <= 1600 else temp_text[:1597] + "..."
        few_shot_text += temp_text

    template = ManualTemplate(tokenizer=tokenizer, text=f'{few_shot_text} {{ "placeholder":"text_a" }} It was {{ "mask" }}')

    # if the total tokens after tokenization would exceed max_seq_length
    total_tokens = len(tokenizer.tokenize(few_shot_text)) + len(tokenizer.tokenize(test_sentence))
    if total_tokens > max_seq_length:
        print(f'Skipping example {idx} as the total token count {total_tokens} exceeds the maximum sequence length {max_seq_length}.')
        continue

    prompt_model = PromptForClassification(plm=plm, template=template, verbalizer=verbalizer, freeze_plm=False)
    test_dataloader = PromptDataLoader(dataset=[test_example], template=template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, max_seq_length=2048, batch_size=16, teacher_forcing=False, predict_eos_token=False, truncate_method='tail')

    with torch.no_grad():
        for inputs in test_dataloader:
            if use_cuda:
                inputs = {k: v.to(device) for k, v in inputs.items() if torch.is_tensor(v)}
            logits = prompt_model(inputs)
            labels = inputs['label']
            preds = torch.argmax(logits, dim=-1)
            allpreds.append(preds[0])
            alllabels.append(labels[0])

# to liat
allpreds = [pred.item() for pred in allpreds]
alllabels = [label.item() for label in alllabels]

# compute accuracy, precision, recall and F1-score
accuracy = accuracy_score(alllabels, allpreds)
precision = precision_score(alllabels, allpreds, average='weighted')
recall = recall_score(alllabels, allpreds, average='weighted')
f1 = f1_score(alllabels, allpreds, average='weighted')

#the classification report
class_report = classification_report(alllabels, allpreds)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('Classification Report:')
print(class_report)


Predicting:   0%|          | 0/12796 [00:00<?, ?batch/s]
tokenizing: 1it [00:00, 243.91it/s]
Predicting:   0%|          | 1/12796 [00:00<2:27:33,  1.45batch/s]
tokenizing: 1it [00:00, 413.03it/s]
Predicting:   0%|          | 2/12796 [00:01<2:21:58,  1.50batch/s]
tokenizing: 1it [00:00, 215.90it/s]
Predicting:   0%|          | 3/12796 [00:01<2:20:09,  1.52batch/s]
tokenizing: 1it [00:00, 276.27it/s]
Predicting:   0%|          | 4/12796 [00:02<2:19:22,  1.53batch/s]
tokenizing: 1it [00:00, 89.08it/s]
Predicting:   0%|          | 5/12796 [00:03<2:21:15,  1.51batch/s]
tokenizing: 1it [00:00, 196.80it/s]
Predicting:   0%|          | 6/12796 [00:03<2:20:12,  1.52batch/s]
tokenizing: 1it [00:00, 196.45it/s]
Predicting:   0%|          | 7/12796 [00:04<2:19:48,  1.52batch/s]
tokenizing: 1it [00:00, 281.80it/s]
Predicting:   0%|          | 8/12796 [00:05<2:19:12,  1.53batch/s]
tokenizing: 1it [00:00, 210.45it/s]
Predicting:   0%|          | 9/12796 [00:05<2:19:08,  1.53batch/s]
tokenizing: 1it [

Skipping example 96 as the total token count 3636 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 214.53it/s]
Predicting:   1%|          | 98/12796 [01:03<1:50:43,  1.91batch/s]
tokenizing: 1it [00:00, 191.55it/s]
Predicting:   1%|          | 99/12796 [01:04<1:57:53,  1.80batch/s]
tokenizing: 1it [00:00, 176.19it/s]
Predicting:   1%|          | 100/12796 [01:04<2:03:27,  1.71batch/s]
tokenizing: 1it [00:00, 158.35it/s]
Predicting:   1%|          | 101/12796 [01:05<2:07:41,  1.66batch/s]
tokenizing: 1it [00:00, 160.05it/s]
Predicting:   1%|          | 102/12796 [01:06<2:10:27,  1.62batch/s]
tokenizing: 1it [00:00, 120.35it/s]
Predicting:   1%|          | 103/12796 [01:06<2:13:07,  1.59batch/s]
tokenizing: 1it [00:00, 292.27it/s]
Predicting:   1%|          | 104/12796 [01:07<2:14:33,  1.57batch/s]
tokenizing: 1it [00:00, 142.56it/s]
Predicting:   1%|          | 105/12796 [01:08<2:15:49,  1.56batch/s]
tokenizing: 1it [00:00, 278.45it/s]
Predicting:   1%|          | 106/12796 [01:08<2:16:00,  1.56batch/s]
tokenizing: 1it [00:00, 134.59it/s]
Predicting:   1%|   

Skipping example 189 as the total token count 3933 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 122.90it/s]
Predicting:   1%|▏         | 191/12796 [02:03<1:49:47,  1.91batch/s]
tokenizing: 1it [00:00, 225.73it/s]
Predicting:   2%|▏         | 192/12796 [02:04<1:56:14,  1.81batch/s]
tokenizing: 1it [00:00, 272.02it/s]
Predicting:   2%|▏         | 193/12796 [02:05<2:01:40,  1.73batch/s]
tokenizing: 1it [00:00, 212.48it/s]
Predicting:   2%|▏         | 194/12796 [02:05<2:05:20,  1.68batch/s]
tokenizing: 1it [00:00, 228.06it/s]
Predicting:   2%|▏         | 195/12796 [02:06<2:08:26,  1.64batch/s]
tokenizing: 1it [00:00, 166.92it/s]
Predicting:   2%|▏         | 196/12796 [02:07<2:10:36,  1.61batch/s]
tokenizing: 1it [00:00, 116.60it/s]
Predicting:   2%|▏         | 197/12796 [02:07<2:12:47,  1.58batch/s]
tokenizing: 1it [00:00, 78.97it/s]
Predicting:   2%|▏         | 198/12796 [02:08<2:14:59,  1.56batch/s]
tokenizing: 1it [00:00, 295.64it/s]
Predicting:   2%|▏         | 199/12796 [02:09<2:15:12,  1.55batch/s]
tokenizing: 1it [00:00, 215.51it/s]
Predicting:   2%|▏ 

...

Skipping example 12672 as the total token count 2104 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 265.43it/s]
Predicting:  99%|█████████▉| 12674/12796 [2:18:04<01:03,  1.93batch/s]
tokenizing: 1it [00:00, 162.99it/s]
Predicting:  99%|█████████▉| 12675/12796 [2:18:05<01:06,  1.81batch/s]
tokenizing: 1it [00:00, 152.97it/s]
Predicting:  99%|█████████▉| 12676/12796 [2:18:05<01:09,  1.73batch/s]
tokenizing: 1it [00:00, 119.40it/s]
Predicting:  99%|█████████▉| 12677/12796 [2:18:06<01:11,  1.67batch/s]
tokenizing: 1it [00:00, 254.73it/s]
Predicting:  99%|█████████▉| 12678/12796 [2:18:07<01:12,  1.63batch/s]
tokenizing: 1it [00:00, 254.39it/s]
Predicting:  99%|█████████▉| 12679/12796 [2:18:07<01:13,  1.60batch/s]
tokenizing: 1it [00:00, 175.52it/s]
Predicting:  99%|█████████▉| 12680/12796 [2:18:08<01:13,  1.58batch/s]
tokenizing: 1it [00:00, 239.35it/s]
Predicting:  99%|█████████▉| 12681/12796 [2:18:08<01:13,  1.56batch/s]
tokenizing: 1it [00:00, 59.36it/s]
Predicting:  99%|█████████▉| 12682/12796 [2:18:09<01:14,  1.53batch/s]
tokenizing: 1it [00:00, 165.83it/s]
P

Skipping example 12754 as the total token count 4816 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 194.25it/s]
Predicting: 100%|█████████▉| 12756/12796 [2:18:57<00:20,  1.92batch/s]
tokenizing: 1it [00:00, 127.49it/s]
Predicting: 100%|█████████▉| 12757/12796 [2:18:58<00:21,  1.80batch/s]
tokenizing: 1it [00:00, 155.47it/s]
Predicting: 100%|█████████▉| 12758/12796 [2:18:58<00:22,  1.73batch/s]
tokenizing: 1it [00:00, 170.06it/s]
Predicting: 100%|█████████▉| 12759/12796 [2:18:59<00:22,  1.67batch/s]
tokenizing: 1it [00:00, 214.67it/s]
Predicting: 100%|█████████▉| 12760/12796 [2:19:00<00:22,  1.63batch/s]
tokenizing: 1it [00:00, 116.69it/s]
Predicting: 100%|█████████▉| 12761/12796 [2:19:00<00:21,  1.60batch/s]
tokenizing: 1it [00:00, 105.45it/s]
Predicting: 100%|█████████▉| 12762/12796 [2:19:01<00:21,  1.58batch/s]
tokenizing: 1it [00:00, 114.21it/s]
Predicting: 100%|█████████▉| 12763/12796 [2:19:02<00:21,  1.56batch/s]
tokenizing: 1it [00:00, 98.43it/s]
Predicting: 100%|█████████▉| 12764/12796 [2:19:02<00:20,  1.55batch/s]
tokenizing: 1it [00:00, 96.72it/s]
Pr

Accuracy: 0.8995924126038564
Precision: 0.898265848758476
Recall: 0.8995924126038564
F1 Score: 0.8989163260112557
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94     11512
           1       0.49      0.47      0.48      1246

    accuracy                           0.90     12758
   macro avg       0.71      0.71      0.71     12758
weighted avg       0.90      0.90      0.90     12758



In [ ]:
allpreds = []
alllabels = []

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

# build the prompt model
prompt_model = PromptForClassification(plm=plm, template=None, verbalizer=verbalizer, freeze_plm=False)
if use_cuda:
    prompt_model = prompt_model.cuda()

max_seq_length = 2048

for idx, test_example in enumerate(tqdm(dataset['test'], desc="Predicting", unit="batch")):
    test_sentence = test_example.text_a
    test_embedding = model.encode([test_sentence])[0]  # Get the embedding of the test_sentence

    # predict the best K for this sentence
    best_k = classifier.predict(test_embedding.reshape(1, -1))[0]

    # find the top K most similar train sentences
    #tree = KDTree(train_embeddings)
    _, top_k_indices = tree.query(test_embedding.reshape(1, -1), k=best_k)

    # get the top K most similar examples
    top_k_train_examples = [dataset['train'][i] for i in top_k_indices[0]]

    # truncate top_k_train_examples and create template, to ensure not exceeed the length
    few_shot_text = ""
    for ex in top_k_train_examples:
        temp_text = f"{ex.text_a} It was {verbalizer.label_words[ex.label][0]}."
        temp_text = temp_text if len(temp_text) <= 1600 else temp_text[:1597] + "..."
        few_shot_text += temp_text

    template = ManualTemplate(tokenizer=tokenizer, text=f'{few_shot_text} {{ "placeholder":"text_a" }} It was {{ "mask" }}')

    # if the total tokens after tokenization would exceed max_seq_length
    total_tokens = len(tokenizer.tokenize(few_shot_text)) + len(tokenizer.tokenize(test_sentence))
    if total_tokens > max_seq_length:
        print(f'Skipping example {idx} as the total token count {total_tokens} exceeds the maximum sequence length {max_seq_length}.')
        continue

    prompt_model = PromptForClassification(plm=plm, template=template, verbalizer=verbalizer, freeze_plm=False)
    test_dataloader = PromptDataLoader(dataset=[test_example], template=template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, max_seq_length=2048, batch_size=16, teacher_forcing=False, predict_eos_token=False, truncate_method='tail')

    with torch.no_grad():
        for inputs in test_dataloader:
            if use_cuda:
                inputs = {k: v.to(device) for k, v in inputs.items() if torch.is_tensor(v)}
            logits = prompt_model(inputs)
            labels = inputs['label']
            preds = torch.argmax(logits, dim=-1)
            allpreds.append(preds[0])
            alllabels.append(labels[0])

# to liat
allpreds = [pred.item() for pred in allpreds]
alllabels = [label.item() for label in alllabels]

# compute accuracy, precision, recall and F1-score
accuracy = accuracy_score(alllabels, allpreds)
precision = precision_score(alllabels, allpreds, average='weighted')
recall = recall_score(alllabels, allpreds, average='weighted')
f1 = f1_score(alllabels, allpreds, average='weighted')

#the classification report
class_report = classification_report(alllabels, allpreds, digits=3)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('Classification Report:')
print(class_report)


Predicting:   0%|          | 0/12796 [00:00<?, ?batch/s]
tokenizing: 1it [00:00, 224.71it/s]
Predicting:   0%|          | 1/12796 [00:00<2:01:19,  1.76batch/s]
tokenizing: 1it [00:00, 390.17it/s]
Predicting:   0%|          | 2/12796 [00:01<1:54:10,  1.87batch/s]
tokenizing: 1it [00:00, 272.36it/s]
Predicting:   0%|          | 3/12796 [00:01<1:51:51,  1.91batch/s]
tokenizing: 1it [00:00, 285.91it/s]
Predicting:   0%|          | 4/12796 [00:02<1:50:45,  1.92batch/s]
tokenizing: 1it [00:00, 74.57it/s]
Predicting:   0%|          | 5/12796 [00:02<1:52:56,  1.89batch/s]
tokenizing: 1it [00:00, 372.43it/s]
Predicting:   0%|          | 6/12796 [00:03<1:52:00,  1.90batch/s]
tokenizing: 1it [00:00, 244.92it/s]
Predicting:   0%|          | 7/12796 [00:03<1:51:17,  1.92batch/s]
tokenizing: 1it [00:00, 399.80it/s]
Predicting:   0%|          | 8/12796 [00:04<1:50:31,  1.93batch/s]
tokenizing: 1it [00:00, 281.63it/s]
Predicting:   0%|          | 9/12796 [00:04<1:50:11,  1.93batch/s]
tokenizing: 1it [

Skipping example 96 as the total token count 3636 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 220.42it/s]
Predicting:   1%|          | 98/12796 [00:50<1:27:43,  2.41batch/s]
tokenizing: 1it [00:00, 157.23it/s]
Predicting:   1%|          | 99/12796 [00:51<1:33:16,  2.27batch/s]
tokenizing: 1it [00:00, 180.02it/s]
Predicting:   1%|          | 100/12796 [00:51<1:37:31,  2.17batch/s]
tokenizing: 1it [00:00, 237.61it/s]
Predicting:   1%|          | 101/12796 [00:52<1:40:39,  2.10batch/s]
tokenizing: 1it [00:00, 159.86it/s]
Predicting:   1%|          | 102/12796 [00:52<1:43:23,  2.05batch/s]
tokenizing: 1it [00:00, 148.09it/s]
Predicting:   1%|          | 103/12796 [00:53<1:45:31,  2.00batch/s]
tokenizing: 1it [00:00, 240.97it/s]
Predicting:   1%|          | 104/12796 [00:53<1:46:33,  1.99batch/s]
tokenizing: 1it [00:00, 133.01it/s]
Predicting:   1%|          | 105/12796 [00:54<1:47:51,  1.96batch/s]
tokenizing: 1it [00:00, 259.26it/s]
Predicting:   1%|          | 106/12796 [00:54<1:47:57,  1.96batch/s]
tokenizing: 1it [00:00, 155.34it/s]
Predicting:   1%|   

Skipping example 189 as the total token count 3933 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 163.43it/s]
Predicting:   1%|▏         | 191/12796 [01:38<1:28:17,  2.38batch/s]
tokenizing: 1it [00:00, 335.76it/s]
Predicting:   2%|▏         | 192/12796 [01:38<1:32:51,  2.26batch/s]
tokenizing: 1it [00:00, 239.21it/s]
Predicting:   2%|▏         | 193/12796 [01:39<1:36:49,  2.17batch/s]
tokenizing: 1it [00:00, 289.64it/s]
Predicting:   2%|▏         | 194/12796 [01:39<1:39:42,  2.11batch/s]
tokenizing: 1it [00:00, 248.55it/s]
Predicting:   2%|▏         | 195/12796 [01:40<1:41:57,  2.06batch/s]
tokenizing: 1it [00:00, 252.58it/s]
Predicting:   2%|▏         | 196/12796 [01:40<1:43:36,  2.03batch/s]
tokenizing: 1it [00:00, 181.82it/s]
Predicting:   2%|▏         | 197/12796 [01:41<1:45:07,  2.00batch/s]
tokenizing: 1it [00:00, 101.90it/s]
Predicting:   2%|▏         | 198/12796 [01:41<1:46:51,  1.97batch/s]
tokenizing: 1it [00:00, 227.16it/s]
Predicting:   2%|▏         | 199/12796 [01:42<1:47:12,  1.96batch/s]
tokenizing: 1it [00:00, 230.41it/s]
Predicting:   2%|▏

...

Skipping example 12672 as the total token count 2104 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 210.39it/s]
Predicting:  99%|█████████▉| 12674/12796 [1:49:03<00:50,  2.40batch/s]
tokenizing: 1it [00:00, 203.58it/s]
Predicting:  99%|█████████▉| 12675/12796 [1:49:04<00:53,  2.26batch/s]
tokenizing: 1it [00:00, 209.08it/s]
Predicting:  99%|█████████▉| 12676/12796 [1:49:04<00:55,  2.17batch/s]
tokenizing: 1it [00:00, 149.03it/s]
Predicting:  99%|█████████▉| 12677/12796 [1:49:05<00:56,  2.10batch/s]
tokenizing: 1it [00:00, 217.13it/s]
Predicting:  99%|█████████▉| 12678/12796 [1:49:05<00:57,  2.05batch/s]
tokenizing: 1it [00:00, 277.07it/s]
Predicting:  99%|█████████▉| 12679/12796 [1:49:06<00:57,  2.02batch/s]
tokenizing: 1it [00:00, 229.16it/s]
Predicting:  99%|█████████▉| 12680/12796 [1:49:06<00:58,  2.00batch/s]
tokenizing: 1it [00:00, 217.77it/s]
Predicting:  99%|█████████▉| 12681/12796 [1:49:07<00:58,  1.98batch/s]
tokenizing: 1it [00:00, 98.22it/s]
Predicting:  99%|█████████▉| 12682/12796 [1:49:07<00:58,  1.96batch/s]
tokenizing: 1it [00:00, 300.26it/s]
P

Skipping example 12754 as the total token count 4816 exceeds the maximum sequence length 2048.



tokenizing: 1it [00:00, 273.12it/s]
Predicting: 100%|█████████▉| 12756/12796 [1:49:45<00:16,  2.40batch/s]
tokenizing: 1it [00:00, 111.10it/s]
Predicting: 100%|█████████▉| 12757/12796 [1:49:46<00:17,  2.25batch/s]
tokenizing: 1it [00:00, 225.31it/s]
Predicting: 100%|█████████▉| 12758/12796 [1:49:46<00:17,  2.16batch/s]
tokenizing: 1it [00:00, 161.34it/s]
Predicting: 100%|█████████▉| 12759/12796 [1:49:47<00:17,  2.09batch/s]
tokenizing: 1it [00:00, 246.97it/s]
Predicting: 100%|█████████▉| 12760/12796 [1:49:47<00:17,  2.05batch/s]
tokenizing: 1it [00:00, 131.62it/s]
Predicting: 100%|█████████▉| 12761/12796 [1:49:48<00:17,  2.01batch/s]
tokenizing: 1it [00:00, 127.52it/s]
Predicting: 100%|█████████▉| 12762/12796 [1:49:48<00:17,  1.98batch/s]
tokenizing: 1it [00:00, 124.59it/s]
Predicting: 100%|█████████▉| 12763/12796 [1:49:49<00:16,  1.96batch/s]
tokenizing: 1it [00:00, 104.74it/s]
Predicting: 100%|█████████▉| 12764/12796 [1:49:49<00:16,  1.94batch/s]
tokenizing: 1it [00:00, 136.14it/s]


Accuracy: 0.8993572660291582
Precision: 0.8982068626822973
Recall: 0.8993572660291582
F1 Score: 0.8987725214274355
Classification Report:
              precision    recall  f1-score   support

           0      0.943     0.946     0.944     11512
           1      0.484     0.472     0.478      1246

    accuracy                          0.899     12758
   macro avg      0.714     0.709     0.711     12758
weighted avg      0.898     0.899     0.899     12758

